In [2]:
import cv2
import sys
import mediapipe as mp
import numpy as np

In [3]:


# Callback function for mouse click event
def close_window(event, x, y, flags, param):
    global close_requested
    if event == cv2.EVENT_LBUTTONDOWN:
        close_requested = True


# Calculate angle
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Variables
close_requested = False

In [4]:
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    joint1 = mp_pose.PoseLandmark.LEFT_HIP.value
    joint2 = mp_pose.PoseLandmark.LEFT_SHOULDER.value
    joint3 = mp_pose.PoseLandmark.LEFT_ELBOW.value

    while True:
        print("Seleccione una opción:")
        print("1. Contar número de repeticiones")
        print("2. Verificar movilidad")
        option = input("Ingrese su opción: ")

        if option == "1":
            max_reps = int(input("Ingrese el número máximo de repeticiones: "))
            rep_count = 0
            rep_started = False
            break

        elif option == "2":
            max_angle = 0
            break

        else:
            print("Opción inválida. Intente de nuevo.")

    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Video")
    cv2.setMouseCallback("Video", close_window)

    while cap.isOpened():
        ret, frame = cap.read()

        # Detect and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Make detection
        image.flags.writeable = False
        results = pose.process(image)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[joint1].x, landmarks[joint1].y]
            elbow = [landmarks[joint2].x, landmarks[joint2].y]
            wrist = [landmarks[joint3].x, landmarks[joint3].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, str(angle),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            if option == "1":
                # Count reps
                if angle > 160 and not rep_started:
                    rep_started = True

                elif angle < 80 and rep_started:
                    rep_count += 1
                    rep_started = False

                cv2.putText(image, "Reps: " + str(rep_count),
                            (10, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

                if rep_count >= max_reps:
                    height = image.shape[0]  # Obtiene la altura de la imagen
                    cv2.putText(image, "Ejercicio completado",
                                (10, height - 10),  # Coordenadas actualizadas para la esquina inferior izquierda
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)  # Color actualizado a verde (BGR)
            elif option == "2":
                max_angle = max(max_angle, angle)
                cv2.putText(image, "Angulo maximo: " + str(max_angle),
                            (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        except:
            pass

        # Render
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        cv2.imshow('Video', image)

        if cv2.waitKey(50) & 0xFF == ord('q') or close_requested:
            break

    cap.release()
    cv2.destroyAllWindows()


Seleccione una opción:
1. Contar número de repeticiones
2. Verificar movilidad
